# 空氣污染監測網 網路爬蟲實作練習


* 能夠利用 selenium + BeautifulSoup 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料
* ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料





### ① 取出 台北市士林區 2018/01 – 2018/08 的 SO2 資料

In [77]:
# 打開瀏覽器

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time

browser = webdriver.Chrome(executable_path='chromedriver')

browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

In [78]:
# 模擬使用者操作行為，選擇/點擊

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')

browser.find_element_by_id('ctl05_btnQuery').click()
time.sleep(1)

In [79]:
# 取得資料，丟到 BeautifulSoup 解析

html_source = browser.page_source

soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')

count = 0
values = []

for tr in table.find_all('tr')[1:]:
    if len(tr.find_all('td')) == 5 and tr.find('td', class_="no-alt").text :
        values.append({
            "Month": tr.find_all('td')[2].text,
            "value": float(tr.find_all('td')[3].text)
        })
    if count < 8 and len(tr.find_all('td')) == 3:
        values.append({
            "Month": tr.find_all('td')[0].text,
            "value": float(tr.find_all('td')[1].text)
        })
    elif count >=8 :
        break
        
    count += 1

In [80]:
import pandas as pd

df = pd.DataFrame(values)
df

,Month,value
0,2018/01,1.8
1,2018/02,1.9
2,2018/03,2.2
3,2018/04,2.3
4,2018/05,3.1
5,2018/06,2.7
6,2018/07,2.2
7,2018/08,2.4


In [81]:
browser.close()

### ② 取出 台北市士林區 2018/01 – 2018/08 的 SO2、CO 資料

In [97]:
# 打開瀏覽器

from selenium import webdriver
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import time
import pandas as pd

browser = webdriver.Chrome(executable_path='chromedriver')

browser.get("http://taqm.epa.gov.tw/taqm/tw/MonthlyAverage.aspx")

In [98]:
# 模擬使用者操作行為，選擇/點擊

selectSite = Select(browser.find_element_by_id("ctl05_ddlSite"))
selectSite.select_by_value('11')
selectYear = Select(browser.find_element_by_id("ctl05_ddlYear"))
selectYear.select_by_value('2018')

browser.find_element_by_id('ctl05_btnQuery').click()
time.sleep(1)

In [99]:
# 取得資料，丟到 BeautifulSoup 解析

html_source = browser.page_source

soup = BeautifulSoup(html_source, 'html.parser')
table = soup.find('table', class_='TABLE_G')
target = ['SO2', 'CO']

count = 0
so2_values = []
co_values = []


for tr in table.find_all('tr')[1:]:
    if len(tr.find_all('td')) == 5 and (tr.find(class_="no-alt").text in target) :
        item = tr.find('td', class_="no-alt").text
        if item == target[0] :
            so2_values.append({
                "Month": tr.find_all('td')[2].text,
                "value": float(tr.find_all('td')[3].text)
            })
        elif item == target[1] :
            co_values.append({
                "Month": tr.find_all('td')[2].text,
                "value": float(tr.find_all('td')[3].text)
            })
            
    if len(tr.find_all('td')) == 3 and count<20:
        if item == target[0] and len(so2_values) < 8 :
            so2_values.append({
                "Month": tr.find_all('td')[0].text,
                "value": float(tr.find_all('td')[1].text)
            })
        elif len(co_values) < 8 and item == target[1] :
            co_values.append({
                "Month": tr.find_all('td')[0].text,
                "value": float(tr.find_all('td')[1].text)
            })
    if count >= 20:
        break
    
    count += 1


In [100]:
df = pd.DataFrame(so2_values)
print("SO2")
print(df)

print()

df = pd.DataFrame(co_values)
print("CO")
print(df)

SO2
     Month  value
0  2018/01    1.8
1  2018/02    1.9
2  2018/03    2.2
3  2018/04    2.3
4  2018/05    3.1
5  2018/06    2.7
6  2018/07    2.2
7  2018/08    2.4

CO
     Month  value
0  2018/01   0.34
1  2018/02   0.44
2  2018/03   0.40
3  2018/04   0.38
4  2018/05   0.34
5  2018/06   0.29
6  2018/07   0.21
7  2018/08   0.30


In [101]:
browser.close()